<font size="12">Import necessary packet</font>

In [54]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime
import plotly.graph_objects as go
import cvxpy as cp
from scipy import sparse

<font size="8">Pull data</font>

In [55]:
Ticker_names = ['AAPL', 'AMD', 'MSFT'] # Names of tickers(in yahoo finance) that we want to pull

<font size="6">Get ticker objects</font>

In [56]:
# Create an empty dictionary to store Yahoo Finance Ticker objects
Ticker = dict()

# Iterate through each ticker name in the list Ticker_names
for ticker_name in Ticker_names:
    
    # Create a new entry in the dictionary where the key is the current ticker_name,
    # and the value is a Yahoo Finance Ticker object for the corresponding financial instrument
    Ticker[ticker_name] = yf.Ticker(ticker_name) 
    
# The dictionary Ticker now contains Yahoo Finance Ticker objects for each ticker name
Ticker


{'AAPL': yfinance.Ticker object <AAPL>,
 'AMD': yfinance.Ticker object <AMD>,
 'MSFT': yfinance.Ticker object <MSFT>}

Pull ticker data by history() method
- interval: data interval (1m data is only for available for last 7 days, and data interval <1d for the last 60 days) Valid intervals are:
“1m”, “2m”, “5m”, “15m”, “30m”, “60m”, “90m”, “1h”, “1d”, “5d”, “1wk”, “1mo”, “3mo”
- start: If not using period – in the format (yyyy-mm-dd) or datetime.
- end: If not using period – in the format (yyyy-mm-dd) or datetime.

In [57]:
# Create an empty dictionary to store historical data for each ticker
Ticker_historical_data = dict()

# Define start and end dates for historical data retrieval
startDate = datetime.datetime(2018, 1, 2)
endDate = datetime.datetime(2021, 12, 31)

# Define the interval for historical data (e.g., daily data)
interval = '1d'

# Iterate through each ticker name in the list Ticker_names
for ticker_name in Ticker_names:
    
    # Retrieve historical data for the current ticker within the specified date range and interval
    historical_data = Ticker[ticker_name].history(start= startDate, end= endDate, interval= interval, prepost= True)[['Close']]
    
    # Rename the 'Close' column to the ticker symbol for clarity
    historical_data.rename(columns={'Close': '%s' % ticker_name}, inplace=True)
    
    # Set the index of the historical data to be the date
    historical_data.index = historical_data.index.date
    
    # Store the historical data for the current ticker in the dictionary
    Ticker_historical_data[ticker_name] = historical_data

# For example, you can access historical data for 'AAPL' using the following:
Ticker_historical_data['AAPL']['AAPL'].tolist()

[40.722877502441406,
 40.715782165527344,
 40.9049072265625,
 41.3706169128418,
 41.21696472167969,
 41.21223449707031,
 41.20277786254883,
 41.43681335449219,
 41.864707946777344,
 41.651939392089844,
 42.33987808227539,
 42.3776969909668,
 42.188575744628906,
 41.84342575073242,
 41.85287857055664,
 41.18622589111328,
 40.45101547241211,
 40.54557418823242,
 39.70634078979492,
 39.47230529785156,
 39.58103561401367,
 39.66379165649414,
 37.942771911621094,
 36.99479293823242,
 38.54087829589844,
 37.71582794189453,
 36.678009033203125,
 37.12664794921875,
 38.622047424316406,
 39.00896072387695,
 39.72818374633789,
 41.06218719482422,
 40.92924499511719,
 40.79158401489258,
 40.60643768310547,
 40.94587707519531,
 41.65798568725586,
 42.48164749145508,
 42.34397506713867,
 42.27988052368164,
 41.5392951965332,
 41.826499938964844,
 41.97129821777344,
 41.935699462890625,
 41.54642105102539,
 41.999778747558594,
 42.721378326416016,
 43.13440704345703,
 42.71900939941406,
 42.35584259

<font size="8">Log Return</font>

\begin{equation}
\text{Log Return} = \ln\left(\frac{S_t}{S_{t-1}}\right)
\end{equation}

Where:
\begin{align*}
\text{Log Return} & \text{ is the logarithmic return of the investment.} \\
S_t & \text{ is the price of the investment at time } t. \\
S_{t-1} & \text{ is the price of the investment at the previous time period, } t-1.
\end{align*}

In [58]:
Price_dataframe = pd.concat(Ticker_historical_data.values(), axis = 1).dropna()

Price_dataframe

,AAPL,AMD,MSFT
2018-01-02,40.722878,10.980000,80.228989
2018-01-03,40.715782,11.550000,80.602379
2018-01-04,40.904907,12.120000,81.311798
2018-01-05,41.370617,11.880000,82.319901
2018-01-08,41.216965,12.280000,82.403908
...,...,...,...
2021-12-23,174.288620,146.139999,328.668762
2021-12-27,178.292877,154.360001,336.289154
2021-12-28,177.264603,153.149994,335.110718
2021-12-29,177.353638,148.259995,335.798187


In [59]:
# Create a shifted DataFrame by shifting Price_dataframe by one period
shifting_dataframe = Price_dataframe.shift(periods= 1).T

# Set the first column of the shifted DataFrame to the first row of Price_dataframe
shifting_dataframe[Price_dataframe.index[0]] = Price_dataframe.iloc[0].tolist()

# Create a DataFrame for log returns by taking the natural logarithm of the ratio
# of Price_dataframe to the shifted DataFrame (to calculate daily log returns)
Log_return_dataframe = pd.DataFrame(
                                    np.log(Price_dataframe.values.T / shifting_dataframe.values),
                                    columns= Price_dataframe.index.tolist(),
                                    index= Price_dataframe.columns.tolist()
                                    ).T

# Log_return_dataframe now contains the calculated log returns for each ticker
Log_return_dataframe


,AAPL,AMD,MSFT
2018-01-02,0.000000,0.000000,0.000000
2018-01-03,-0.000174,0.050610,0.004643
2018-01-04,0.004634,0.048172,0.008763
2018-01-05,0.011321,-0.020001,0.012322
2018-01-08,-0.003721,0.033116,0.001020
...,...,...,...
2021-12-23,0.003637,0.015585,0.004462
2021-12-27,0.022715,0.054722,0.022921
2021-12-28,-0.005784,-0.007870,-0.003510
2021-12-29,0.000502,-0.032450,0.002049


<font size="8">Mean and Covariance</font>

In [60]:
# Calculate the mean of log returns for each ticker and annualize by multiplying by 30 (assuming 30 trading days in a month)
Mean = np.asarray(np.mean(Log_return_dataframe.values.T, axis= 1)) * 30

# Calculate the covariance matrix of log returns for the tickers and annualize by multiplying by 30
Cov = np.asmatrix(np.cov(Log_return_dataframe.values.T)) * 30

print('Mean array is',Mean)
print('Covariance matrix is',Cov)

Mean array is [0.04363721 0.07691238 0.04242019]
Covariance matrix is [[0.01289867 0.01113776 0.00888883]
 [0.01113776 0.03626638 0.01056734]
 [0.00888883 0.01056734 0.01061027]]


In [61]:
# Set the number of scenarios
scenarios = 10000

# Get the number of assets (tickers)
number_of_assets = len(Ticker_names)

# Generate random samples from a multivariate normal distribution
expected_return_simulation = np.random.multivariate_normal(np.zeros(number_of_assets), Cov, size= scenarios)

# Shift the generated samples to match the specified mean values
expected_return_simulation = expected_return_simulation + Mean

# The resulting expected_return_simulation array contains 10,000 scenarios of returns for the given assets,
# where each row represents a scenario and each column represents the return for a specific asset.
expected_return_simulation

array([[ 0.01969241, -0.17840132,  0.09228174],
       [-0.06305162,  0.10570977,  0.00842323],
       [ 0.23958826,  0.40330699,  0.05636818],
       ...,
       [-0.06526059, -0.11942398, -0.03368852],
       [ 0.07843501, -0.19320894,  0.01418186],
       [ 0.22349571,  0.33250241,  0.17461566]])

<font size="8">Example portfolio</font>

In [62]:
# Define portfolio proportions for each asset in the portfolio
portfolio_proportion = [2/3, 1/6, 1/6]

# Specify the confidence level for Value at Risk (VaR) calculation
confidence_level = 0.95

# Calculate the portfolio payoffs by taking the sum of the element-wise product
# of portfolio_proportion and expected_return_simulation for each scenario
payoffs = sum((portfolio_proportion * expected_return_simulation).T)

# Calculate the Value at Risk (VaR) at the specified confidence level
VaR = np.percentile(payoffs, (1 - confidence_level) * 100)

# Identify returns below the VaR threshold to calculate Conditional Value at Risk (CVaR)
lower_returns = [i for i in payoffs if i <= VaR]

# Calculate the Conditional Value at Risk (CVaR) by taking the mean of the returns below the VaR threshold
CVaR = -1 * np.mean(lower_returns)

# Print the results
print('The portfolio proportion is ', dict(zip(Ticker_names,portfolio_proportion)))
print('Mean of return is %f' % np.mean(payoffs))
print('VaR is %f' % -VaR)  # Note: Multiplying by -1 to present the VaR as a positive value
print('CVaR is %f' % CVaR)


The portfolio proportion is  {'AAPL': 0.6666666666666666, 'AMD': 0.16666666666666666, 'MSFT': 0.16666666666666666}
Mean of return is 0.048524
VaR is 0.130557
CVaR is 0.175920


<font size="8">Plotting histogram</font>

In [63]:
# Specify the bin size for the histogram
size = 0.01

# Create a histogram using Plotly (go.Histogram)
hist = go.Figure(data=[go.Histogram(x=payoffs, xbins=dict(
    start=min(payoffs) // size * size,
    end=max(payoffs) // size * size + size,
    size=size
))])

# Update layout settings for the histogram
hist.update_layout(
    title_text='Portfolio payoffs with mean of return are %f' % np.mean(payoffs),
    xaxis_title_text='Return',
    yaxis_title_text='Count',
    bargroupgap=0.1  # gap between bars of the same location coordinates
)

# Identify returns within the VaR range for highlighting
y_high = [i for i in payoffs if (i >= VaR // size * size and i <= VaR // size * size + size)]

# Add a red line representing the VaR threshold
hist.add_shape(
    dict(
        type="line",
        x0=VaR,
        x1=VaR,
        y0=0,
        y1=len(y_high),
        line=dict(
            color="red",  # Color of the line
            width=2  # Line width
        )
    )
)

# Add an arrow annotation pointing at the VaR line
arrow_text = 'VaR is %f' % VaR
hist.add_annotation(
    text=arrow_text,
    x=VaR,
    y=len(y_high),  # Adjust the vertical position of the arrow text
    showarrow=True,
    arrowhead=2,  # Arrowhead style (2: arrowhead at the end of the line)
    arrowwidth=2,  # Width of the arrowhead
    arrowcolor="blue",  # Color of the arrowhead
    ax=-30  # Adjust the x position of the arrow
)

# Show the histogram
hist.show()


<font size="8">CVaR optimization</font>

<font size="6">Minimization in Linear programing</font>

\begin{align*}
\text{minimize:} & \quad c^T x, \\
\text{subject to:} & \quad A_{ub} x \leq b_{ub}, \\
 & \quad A_{eq} x = b_{eq}, \\
 & \quad lb \leq x \leq ub,
\end{align*}

Where:
\begin{align*}
x & \text{ is the vector of decision variables.} \\
c & \text{ is the coefficient vector of the objective function.} \\
A_{ub} & \text{ is the matrix of coefficients for the upper-bound inequality constraints.} \\
b_{ub} & \text{ is the right-hand side vector of the upper-bound inequality constraints.} \\
A_{eq} & \text{ is the matrix of coefficients for the equality constraints.} \\
b_{eq} & \text{ is the right-hand side vector of the equality constraints.} \\
l & \text{ is the vector of lower bounds on the decision variables.} \\
u & \text{ is the vector of upper bounds on the decision variables.}
\end{align*}

<font size="6">CVaR minimize equation</font>

\begin{array}{rrclr}
\text{minimize: } & \alpha + \cfrac{1}{q(1-\beta)} \sum_{k=1}^{q}u_k \\
\text{subject to: } & -x^Ty_k- \alpha -u_k & \leq & 0 & \text{for } k=1,2,...,q,  \\
 & \sum x & = & 1, \\
 & x^T \bar{y} & = & R, \\
 & u_k & \geq & 0 & \text{for } k = 1,2,...,q, \\
 & x,\alpha & \in & \mathbb{R}
\end{array}

Where:

\begin{align*}
x & = \begin{bmatrix}
                                x^1 \\
                                x^2 \\
                                \vdots \\
                                x^n
            \end{bmatrix},y =  \begin{bmatrix}
            y^1_1 & y^1_2 & ... & y^1_q \\
            y^2_1 & y^2_2 & ... & y^2_q \\
            \vdots & \vdots & \ddots & \vdots &  \\
            y^n_1 & y^n_2 & ... & y^n_q &  \\
            \end{bmatrix},y_k = \begin{bmatrix}
                                y^1_k \\
                                y^2_k \\
                                \vdots \\
                                y^n_k
            \end{bmatrix},\bar{y} = \begin{bmatrix}
                                \bar{y}^1 \\
                                \bar{y}^2 \\
                                \vdots \\
                                \bar{y}^n
            \end{bmatrix}, \alpha, u_k \in \mathbb{R} & \text{for } k = 1,2,...,q\\\\
y^i_j & \text{ is a return of the } i^{th} \text{ asset of the } j^{th} \text{ scenario}, \text{and } \bar{y}^i \text{ is an average return of the } i^{th} \text{asset.}
\end{align*}


In [64]:
# Define variables
beta = confidence_level  # Confidence level for CVaR optimization
q = scenarios  # Number of scenarios
require_return = np.mean(payoffs)  # Required return for the portfolio (mean of simulated returns)
short_sell = None  # Variable indicating whether short selling is allowed

y = expected_return_simulation  # Matrix of simulated expected returns for each asset in each scenario
y_bar = np.mean(expected_return_simulation, axis=0)  # Mean of expected returns across all scenarios for each asset

X = cp.Variable(number_of_assets)  # Portfolio weight variables
alpha = cp.Variable()  # CVaR level variable
u = cp.Variable(q)  # Auxiliary variable for CVaR constraints

# Define the objective function for CVaR minimization
objective = cp.Minimize(alpha + 1/(q*(1-beta)) * cp.sum(u))

# Define the constraints for CVaR optimization
constraints = [
    -1 * X.T @ y[i] - alpha - u[i] <= 0 for i in range(q)
] + [
    cp.sum(X) == 1,  # Portfolio weights sum to 1
    X.T @ y_bar == require_return,  # Portfolio expected return constraint
    u >= np.zeros(q)  # Non-negativity constraints for auxiliary variable
]

if short_sell == None:
    constraints = constraints + [X >= np.zeros(number_of_assets)]

# Formulate the CVaR optimization problem
problem = cp.Problem(objective, constraints)

# Solve the CVaR optimization problem
problem.solve()

# Extract the optimized values
Cvar = problem.value  # Optimized CVaR value
x = X.value  # Optimized portfolio weights
Var = alpha.value  # Optimized VaR level


<font size="6">Convert to matrix form</font>

\begin{align*}
\text{minimize:} & \quad f^T z \\
\text{subject to:} & \quad A \times z \leq b, \\
 & \quad A_{eq} \times z = b_{eq}, \\
 & \quad lb \leq z \leq ub.
\end{align*}

Where:

\begin{align*}
z^T & =    \begin{bmatrix}
            x_1 & x_2 & ... & x_n & \alpha & u_1 & u_2 & ... & u_n
            \end{bmatrix}, \\
f^T & =    \begin{bmatrix}
            0 & 0 & ... & 0 & 1 & \cfrac{1}{q(1-\beta)} & \cfrac{1}{q(1-\beta)} & ... & \cfrac{1}{q(1-\beta)}
            \end{bmatrix}, \\
A & =  \begin{bmatrix}
            y^1_1 & y^2_1 & ... & y^n_1 & 1 & 1 & 0 & ... & 0 \\
            y^1_2 & y^2_2 & ... & y^n_2 & 1 & 0 & 1 & ... & 0 \\
            \vdots & \vdots & \ddots & \vdots & \vdots & \vdots & \vdots & \ddots & \vdots \\
            y^1_q & y^2_q & ... & y^n_q & 1 & 1 & 0 & ... & 0 \\
            \end{bmatrix}, b = \begin{bmatrix}
                                0 \\
                                0 \\
                                \vdots \\
                                0
            \end{bmatrix},\\
A_{eq} & =  \begin{bmatrix}
            1 & 1 & ... & 1 & 0 & 0 & 0 & ... & 0 \\
            \bar{y}^1 & \bar{y}^2 & ... & \bar{y}^n & 0 & 0 & 0 & ... & 0 \\
            \end{bmatrix} \text{ and } b_{eq} = \begin{bmatrix}
                                                1 \\
                                                R
                                                \end{bmatrix} \\
y^i_j & \text{ is a return of the } i^{th} \text{ asset of the } j^{th} \text{ scenario}, \text{and } \bar{y}^i \text{ is an average return of the } i^{th} \text{asset.}
\end{align*} \\



In [65]:
# Define variables for CVaR optimization
beta = confidence_level  # Confidence level for CVaR optimization
q = scenarios  # Number of scenarios
len_of_vector_z = number_of_assets + 1 + q  # Length of the decision variable vector z
mean_of_expected_return_simulation_vector = np.mean(expected_return_simulation, axis=0)  # Mean of expected returns across all scenarios for each asset
require_return = np.mean(payoffs)  # Required return for the portfolio 
short_sell = None  # Variable indicating whether short selling is allowed

# Define the decision variable vector z
z = cp.Variable(len_of_vector_z)

# Initialize the objective function coefficients
f = np.zeros(len_of_vector_z)
f[number_of_assets] = 1
f[number_of_assets + 1:] = 1 / (q * (1 - beta))

# Define the objective function for CVaR minimization
objective = cp.Minimize(f.T @ z)

# Define the constraint matrix A for inequality constraints
vector_one = np.array([[1 for i in range(q)]])
identity_matrix = np.identity(q)
A = -1 * np.concatenate((expected_return_simulation, vector_one.T, identity_matrix), axis=1)

# Define the right-hand side of the inequality constraints
b = np.zeros(q)

# Define the equality constraint matrix Aeq
Aeq = np.zeros((2, len_of_vector_z))
Aeq[0, :number_of_assets] = 1
Aeq[1, :number_of_assets] = mean_of_expected_return_simulation_vector

# Define the right-hand side of the equality constraints
beq = np.ones(2)
beq[1] = require_return

# Convert A and Aeq to sparse matrices for efficient processing
A = sparse.csc_matrix(np.asmatrix(A))
Aeq = sparse.csc_matrix(np.asmatrix(Aeq))

# Define the constraints for CVaR optimization
constraints = [
    A @ z <= b,
    Aeq @ z == beq,
    z[number_of_assets:] >= np.zeros(len_of_vector_z - number_of_assets)
]

if short_sell == None:
    constraints += [z[0:number_of_assets] >= np.zeros(number_of_assets)]

# Formulate the CVaR optimization problem
problem = cp.Problem(objective, constraints)

# Solve the CVaR optimization problem
problem.solve()

# Extract the optimized values
Cvar = problem.value  # Optimized CVaR value
x = z[0:number_of_assets].value  # Optimized portfolio weights
Var = z[number_of_assets].value  # Optimized VaR level


<font size="8">Optimal portfolio</font>

In [66]:
# Calculate portfolio payoffs using the optimal portfolio proportions
optimal_portfolio_proportion = x
confidence_level = 0.95

# Calculate portfolio payoffs using the optimal portfolio proportions and Monte Carlo simulation
payoffs_o = sum((optimal_portfolio_proportion * expected_return_simulation).T)

# Calculate Value at Risk (VaR) using the specified confidence level
VaR_o = np.percentile(payoffs_o, (1 - confidence_level) * 100)

# Identify returns below the VaR threshold to calculate Conditional Value at Risk (CVaR)
lower_returns_o = [i for i in payoffs_o if i <= VaR_o]

# Calculate Conditional Value at Risk (CVaR) using the mean of returns below the VaR threshold
CVaR_o = -1 * np.mean(lower_returns_o)

# Print the results
print('The optimal portfolio proportion is ', dict(zip(Ticker_names,optimal_portfolio_proportion)))
print('Mean of return is %f' % np.mean(payoffs_o))
print('VaR is %f' % -VaR_o)
print('VaR from optimization is %f' % Var)
print('CVaR is %f' % CVaR_o)
print('CVaR from optimization is %f' % Cvar)

The optimal portfolio proportion is  {'AAPL': 0.2273955105039571, 'AMD': 0.16069698572724306, 'MSFT': 0.6119075037688012}
Mean of return is 0.048524
VaR is 0.119380
VaR from optimization is 0.119380
CVaR is 0.165062
CVaR from optimization is 0.165062


<font size="8">Plotting histogram</font>

In [67]:
# Create a histogram using Plotly (go.Histogram) for the optimal portfolio payoffs
hist = go.Figure(data=[go.Histogram(x=payoffs_o, xbins=dict(
    start=min(payoffs_o) // size * size,  
    end=max(payoffs_o) // size * size + size,  
    size=size
))])

# Update layout settings for the histogram
hist.update_layout(
    title_text='Portfolio payoffs with mean of return are %f' % np.mean(payoffs_o),  # title of plot
    xaxis_title_text='Return',  # x-axis label
    yaxis_title_text='Count',  # y-axis label
    bargroupgap=0.1  # gap between bars of the same location coordinates
)

# Identify returns within the VaR range for highlighting
y_high = [i for i in payoffs_o if (i >= VaR_o // size * size and i <= VaR_o // size * size + size)]  

# Add a red line representing the VaR threshold for the optimal portfolio
hist.add_shape(
    dict(
        type="line",
        x0=VaR_o,
        x1=VaR_o,
        y0=0,
        y1=len(y_high),
        line=dict(
            color="red",  # Color of the line
            width=2  # Line width
        )
    )
)

# Add an arrow annotation pointing at the VaR line for the optimal portfolio
arrow_text = 'VaR is %f' % VaR_o
hist.add_annotation(
    text=arrow_text,
    x=VaR_o,
    y=len(y_high),  # Adjust the vertical position of the arrow text
    showarrow=True,
    arrowhead=2,  # Arrowhead style (2: arrowhead at the end of the line)
    arrowwidth=2,  # Width of the arrowhead
    arrowcolor="blue",  # Color of the arrowhead
    ax=-30
)

# Show the histogram for the optimal portfolio
hist.show()


In [68]:
# Create a histogram using Plotly (go.Histogram) for the optimal portfolio payoffs
hist = go.Figure(data=[go.Histogram(x=payoffs_o, xbins=dict(
    start=min(payoffs_o) // size * size,  # type: ignore
    end=max(payoffs_o) // size * size + size,  # type: ignore
    size=size
), name='Optimal portfolio with CVaR is %f' % CVaR_o)])

# Update layout settings for the histogram
hist.update_layout(
    title_text='Portfolios payoffs with mean of return are %f' % np.mean(payoffs_o),  # title of plot
    xaxis_title_text='Return',  # x-axis label
    yaxis_title_text='Count',  # y-axis label
    bargroupgap=0.1  # gap between bars of the same location coordinates
)

# Identify returns within the VaR range for highlighting
y_high = [i for i in payoffs_o if (i >= VaR_o // size * size and i <= VaR_o // size * size + size)]  # type: ignore

# Add a blue line representing the VaR threshold for the optimal portfolio
hist.add_shape(
    dict(
        type="line",
        x0=VaR_o,
        x1=VaR_o,
        y0=0,
        y1=len(y_high),
        line=dict(
            color="blue",  # Color of the line
            width=2  # Line width
        )
    )
)

# Add an arrow annotation pointing at the VaR line for the optimal portfolio
arrow_text = 'VaR is %f' % VaR_o
hist.add_annotation(
    text=arrow_text,
    x=VaR_o,
    y=len(y_high),  # Adjust the vertical position of the arrow text
    showarrow=True,
    arrowhead=2,  # Arrowhead style (2: arrowhead at the end of the line)
    arrowwidth=2,  # Width of the arrowhead
    arrowcolor="blue",  # Color of the arrowhead
    ax=-30
)

# Add a histogram for the random portfolio payoffs
hist.add_trace(go.Histogram(x=payoffs, xbins=dict(
    start=min(payoffs) // size * size,  # type: ignore
    end=max(payoffs) // size * size + size,  # type: ignore
    size=size
), name='Random portfolio with CVaR is %f' % CVaR))

# Identify returns within the VaR range for highlighting in the random portfolio
y_high = [i for i in payoffs if (i >= VaR // size * size and i <= VaR // size * size + size)]  # type: ignore

# Add a red line representing the VaR threshold for the random portfolio
hist.add_shape(
    dict(
        type="line",
        x0=VaR,
        x1=VaR,
        y0=0,
        y1=len(y_high),
        line=dict(
            color="red",  # Color of the line
            width=2  # Line width
        )
    )
)

# Add an arrow annotation pointing at the VaR line for the random portfolio
arrow_text = 'VaR is %f' % VaR
hist.add_annotation(
    text=arrow_text,
    x=VaR,
    y=len(y_high),  # Adjust the vertical position of the arrow text
    showarrow=True,
    arrowhead=2,  # Arrowhead style (2: arrowhead at the end of the line)
    arrowwidth=2,  # Width of the arrowhead
    arrowcolor="red",  # Color of the arrowhead
    ax=-180
)

# The two histograms are drawn on top of one another
hist.update_layout(barmode='overlay')
hist.update_traces(opacity=0.5)

# Show the overlaid histogram
hist.show()


<font size="8">Efficient frontier</font>

In [69]:
# Initialize lists to store CVaR values and feasible returns
CVaR_dict = dict()
Return = [i * 0.001 for i in range(43, 70)]  # Adjusted return values

for i in Return:
    
    # Define variables for CVaR optimization
    beta = confidence_level  # Confidence level for CVaR optimization
    q = scenarios  # Number of scenarios
    len_of_vector_z = number_of_assets + 1 + q  # Length of the decision variable vector z
    mean_of_expected_return_simulation_vector = np.mean(expected_return_simulation, axis=0)  # Mean of expected returns across all scenarios for each asset
    require_return = i  # Required return for the portfolio 
    short_sell = None  # Variable indicating whether short selling is allowed

    # Define the decision variable vector z
    z = cp.Variable(len_of_vector_z)

    # Initialize the objective function coefficients
    f = np.zeros(len_of_vector_z)
    f[number_of_assets] = 1
    f[number_of_assets + 1:] = 1 / (q * (1 - beta))

    # Define the objective function for CVaR minimization
    objective = cp.Minimize(f.T @ z)

    # Define the constraint matrix A for inequality constraints
    vector_one = np.array([[1 for i in range(q)]])
    identity_matrix = np.identity(q)
    A = -1 * np.concatenate((expected_return_simulation, vector_one.T, identity_matrix), axis=1)

    # Define the right-hand side of the inequality constraints
    b = np.zeros(q)

    # Define the equality constraint matrix Aeq
    Aeq = np.zeros((2, len_of_vector_z))
    Aeq[0, :number_of_assets] = 1
    Aeq[1, :number_of_assets] = mean_of_expected_return_simulation_vector

    # Define the right-hand side of the equality constraints
    beq = np.ones(2)
    beq[1] = require_return

    # Convert A and Aeq to sparse matrices for efficient processing
    A = sparse.csc_matrix(np.asmatrix(A))
    Aeq = sparse.csc_matrix(np.asmatrix(Aeq))

    # Define the constraints for CVaR optimization
    constraints = [
        A @ z <= b,
        Aeq @ z == beq,
        z[number_of_assets:] >= np.zeros(len_of_vector_z - number_of_assets)
    ]

    if short_sell == None:
        constraints += [z[0:number_of_assets] >= np.zeros(number_of_assets)]

    # Formulate the CVaR optimization problem
    problem = cp.Problem(objective, constraints)

    # Solve the CVaR optimization problem
    problem.solve()
    
    if problem.status == 'optimal':
        
        CVaR_dict[i] = problem.value

In [70]:
CVaR_list = list(CVaR_dict.values())
feasible_return = list(CVaR_dict.keys())

# Create a scatter plot using Plotly (go.Scatter) for the efficient frontier
fig = go.Figure(data=go.Scatter(x=CVaR_list, y=feasible_return, name='The efficient frontier'))

# Update layout settings for the scatter plot
fig.update_layout(
    title_text='The efficient frontier',  # title of plot
    xaxis_title_text='Risk (CVaR)',  # x-axis label
    yaxis_title_text='Return',  # y-axis label
    bargroupgap=0.1  # gap between bars of the same location coordinates
)

# Add a scatter point for the random portfolio on the efficient frontier plot
fig.add_trace(go.Scatter(x=[CVaR], y=[np.mean(payoffs)], name='Random portfolio'))

# Show the plot
fig.show()


<font size="8">CVaR optimization: Stocks and Cryptocurrencies </font>

In [71]:
def pull_data(Ticker_names, startDate, endDate, interval):
    """The pulling data function from Yahoo finance.

    Args:
        Ticker_names (list): List of ticker names in Yahoo finance that you want to pull.
        startDate (datetime): Starting date of data you want to pull in datetime form.
        endDate (datetime): Ending date of data you want to pull in datetime form.
        interval (str): Data interval. Valid intervals are:
                       “1m”, “2m”, “5m”, “15m”, “30m”, “60m”, “90m”, “1h”, “1d”, “5d”, “1wk”, “1mo”, “3mo”

    Returns:
        DataFrame: Close_price_dataframe
    """    
    # Create dictionaries to store ticker objects and historical data
    Ticker = dict()
    Ticker_historical_data = dict()
    
    # Loop through ticker names to get ticker objects
    for ticker_name in Ticker_names:
        Ticker[ticker_name] = yf.Ticker(ticker_name)  # Get ticker object
        
    # Loop through ticker names to get historical data
    for ticker_name in Ticker_names:
        # Get historical data for each ticker
        Ticker_historical_data[ticker_name] = Ticker[ticker_name].history(
            start=startDate, end=endDate, interval=interval
        )[['Close']]
        # Rename the 'Close' column to the ticker name
        Ticker_historical_data[ticker_name].rename(columns={'Close': '%s' % ticker_name}, inplace=True)
        # Set the index to the date
        Ticker_historical_data[ticker_name].index = Ticker_historical_data[ticker_name].index.date
    
    # Concatenate the historical data for all tickers and drop rows with missing values
    return pd.concat(Ticker_historical_data.values(), axis=1).dropna()

def Log_return(Price_dataframe):
    """Compute the log return from an asset price dataframe.

    Args:
        Price_dataframe (DataFrame): Column: ticker name, Index: date

    Returns:
        DataFrame: Log_return_dataframe
    """        
    # Create a shifted DataFrame by shifting Price_dataframe by one period
    dummy = Price_dataframe.shift(periods=1).T
    
    # Set the first column of the shifted DataFrame to the first row of Price_dataframe
    dummy[Price_dataframe.index[0]] = Price_dataframe.iloc[0].tolist()
    
    # Create a DataFrame for log returns by taking the natural logarithm of the ratio
    # of Price_dataframe to the shifted DataFrame (to calculate daily log returns)
    return pd.DataFrame(
        np.log(Price_dataframe.values.T / dummy.values),
        columns=Price_dataframe.index.tolist(),
        index=Price_dataframe.columns.tolist()
    ).T
    
def Mean_Cov(Log_return_dataframe, shifting_day):
    """Compute the Mean and Covariance of each asset.

    Args:
        Log_return_dataframe (DataFrame): Column: ticker name, Index: date
        shifting_day (int): The number of days to shift.

    Returns:
        (Mean, Cov): Mean list, Covariance matrix
    """    
    # Compute the mean of log returns for each asset and multiply by the shifting days
    Mean = np.asarray(np.mean(Log_return_dataframe.values.T, axis=1)) * shifting_day
    
    # Compute the covariance matrix of log returns and multiply by the shifting days
    Cov = np.asmatrix(np.cov(Log_return_dataframe.values.T)) * shifting_day
    
    return Mean, Cov

def Multivariate_simulation(scenario, Mean, Cov):
    """Simulate the expected return of each asset.

    Args:
        scenario (int): Number of scenarios to simulate.
        Mean (1-D array): Mean list.
        Cov (n-D array): Covariance matrix.

    Returns:
        n-D array: Matrix of scenarios of the expected return of each asset.
    """    
    number_of_assets = len(Mean)
    
    # Generate random samples from a multivariate normal distribution
    mean_simulation = np.random.multivariate_normal(np.zeros(number_of_assets), Cov, size=scenario)
    
    # Shift the simulated values by the mean to obtain scenarios of expected returns
    return mean_simulation + Mean

def Multivariate_simulation_with_symmetry(scenario, Mean, Cov, number_of_assets):
    """Simulate the expected return of each asset with symmetry.

    Args:
        scenario (int): Number of scenarios to simulate.
        Mean (1-D array): Mean list.
        Cov (n-D array): Covariance matrix.
        number_of_assets (int): Number of assets.

    Returns:
        n-D array: Matrix of scenarios of the expected return of each asset.
    """    
    # Generate random samples from a multivariate normal distribution for half the scenarios
    mean_simulation = np.random.multivariate_normal(np.zeros(number_of_assets), Cov, size=int(scenario/2))
    
    # Create the symmetric set of scenarios by concatenating the negative of the first half
    mean_simulation = np.concatenate((mean_simulation, -1 * mean_simulation), axis=0)
    
    # Shift the simulated values by the mean to obtain scenarios of expected returns
    return mean_simulation + Mean

def payoff(portfolio_proportion, confidence_level, expected_return_simulation):
    """Compute payoff, VaR, and CVaR.

    Args:
        portfolio_proportion (1-D array): Array of proportions for each asset.
        confidence_level (float): Confidence level (0.00 - 1.00).
        expected_return_simulation (n-D array): Matrix of scenarios of the expected return of each asset.

    Returns:
        (payoffs, VaR, CVaR): (1-D array, float, float)
    """    
    # Calculate the portfolio payoffs by summing the product of proportions and expected returns
    payoffs = sum((portfolio_proportion * expected_return_simulation).T)
    
    # Calculate VaR (Value at Risk) as the specified percentile of the payoffs
    VaR = np.percentile(payoffs, (1 - confidence_level) * 100)
    
    # Identify lower returns below VaR for CVaR calculation
    lower_returns = [i for i in payoffs if i <= VaR]
    
    # Calculate CVaR (Conditional Value at Risk) as the mean of lower returns
    CVaR = -1 * np.mean(lower_returns)
    
    return payoffs, VaR, CVaR

def CVaR_optimization_with_Monte_Carlo_approximation(confidence_level, expected_return_simulation, require_return, short_sell=None):
    """
    Perform CVaR optimization using Monte Carlo approximation.

    Args:
        confidence_level (float): Confidence level for CVaR optimization.
        expected_return_simulation (n-D array): Matrix of scenarios of the expected return of each asset.
        require_return (float): Required return for the portfolio.
        short_sell (str): If short selling is allowed, input 'yes'.

    Returns:
        tuple: A tuple containing optimization status, objective value, optimized portfolio weights, and optimized VaR level.
    """
    
    # Define variables for CVaR optimization
    beta = confidence_level  # Confidence level for CVaR optimization
    number_of_assets = len(expected_return_simulation[0])
    q = len(expected_return_simulation)  # Number of scenarios
    len_of_vector_z = number_of_assets + 1 + q  # Length of the decision variable vector z
    mean_of_expected_return_simulation_vector = np.mean(expected_return_simulation, axis=0)  # Mean of expected returns across all scenarios for each asset

    # Define the decision variable vector z
    z = cp.Variable(len_of_vector_z)

    # Initialize the objective function coefficients
    f = np.zeros(len_of_vector_z)
    f[number_of_assets] = 1
    f[number_of_assets + 1:] = 1 / (q * (1 - beta))

    # Define the objective function for CVaR minimization
    objective = cp.Minimize(f.T @ z)

    # Define the constraint matrix A for inequality constraints
    vector_one = np.array([[1 for i in range(q)]])
    identity_matrix = np.identity(q)
    A = -1 * np.concatenate((expected_return_simulation, vector_one.T, identity_matrix), axis=1)

    # Define the right-hand side of the inequality constraints
    b = np.zeros(q)

    # Define the equality constraint matrix Aeq
    Aeq = np.zeros((2, len_of_vector_z))
    Aeq[0, :number_of_assets] = 1
    Aeq[1, :number_of_assets] = mean_of_expected_return_simulation_vector

    # Define the right-hand side of the equality constraints
    beq = np.ones(2)
    beq[1] = require_return

    # Convert A and Aeq to sparse matrices for efficient processing
    A = sparse.csc_matrix(np.asmatrix(A))
    Aeq = sparse.csc_matrix(np.asmatrix(Aeq))

    # Define the constraints for CVaR optimization
    constraints = [
        A @ z <= b,
        Aeq @ z == beq,
        z[number_of_assets:] >= np.zeros(len_of_vector_z - number_of_assets)
    ]

    if short_sell == None:
        constraints += [z[0:number_of_assets] >= np.zeros(number_of_assets)]

    # Formulate the CVaR optimization problem
    problem = cp.Problem(objective, constraints)

    # Solve the CVaR optimization problem
    problem.solve()
    
    # Return optimization status, objective value, optimized portfolio weights, and optimized VaR level
    return problem.status, problem.value, z[0:number_of_assets].value, z[number_of_assets].value


def CVaR_efficient_forntier(step, start, end, confidence_level, expected_return_simulation, short_sell = None):
    
    # Initialize lists to store CVaR values and feasible returns
    CVaR_dict = dict()
    Return = [i * step for i in range(start, end)]  # Adjusted return values
    
    for i in Return:
        
        # Call the CVaR optimization function to find the optimal portfolio for each required return
        status, CVaR, __, __ = CVaR_optimization_with_Monte_Carlo_approximation(confidence_level, expected_return_simulation, i, short_sell)
        
        if status == 'optimal':
            
            CVaR_dict[i] = CVaR
            
    return CVaR_dict

In [72]:
Ticker_names = ['AAPL', 'AMD', 'BTC-USD']
startDate = datetime.datetime(2018, 1, 2)
endDate = datetime.datetime(2021, 12, 31)
interval = '1d'

# Call the pull_data function to retrieve historical close prices for the specified tickers and date range
Close_price_dataframe = pull_data(Ticker_names, startDate, endDate, interval)

# Display the resulting dataframe containing historical close prices
Close_price_dataframe

,AAPL,AMD,BTC-USD
2018-01-02,40.722878,10.980000,14982.099609
2018-01-03,40.715778,11.550000,15201.000000
2018-01-04,40.904911,12.120000,15599.200195
2018-01-05,41.370613,11.880000,17429.500000
2018-01-08,41.216969,12.280000,15170.099609
...,...,...,...
2021-12-23,174.288620,146.139999,50784.539062
2021-12-27,178.292892,154.360001,50640.417969
2021-12-28,177.264618,153.149994,47588.855469
2021-12-29,177.353592,148.259995,46444.710938


In [73]:
Log_return_datafrme = Log_return(Close_price_dataframe)
Log_return_datafrme

,AAPL,AMD,BTC-USD
2018-01-02,0.000000,0.000000,0.000000
2018-01-03,-0.000174,0.050610,0.014505
2018-01-04,0.004634,0.048172,0.025858
2018-01-05,0.011321,-0.020001,0.110945
2018-01-08,-0.003721,0.033116,-0.138838
...,...,...,...
2021-12-23,0.003637,0.015585,0.043382
2021-12-27,0.022715,0.054722,-0.002842
2021-12-28,-0.005784,-0.007870,-0.062151
2021-12-29,0.000502,-0.032450,-0.024336


In [74]:
shifting_day = 30

Mean, Cov = Mean_Cov(Log_return_datafrme, shifting_day)
print('Mean array is',Mean)
print('Covariance matrix is',Cov)

Mean array is [0.04363721 0.07691238 0.03417301]
Covariance matrix is [[0.01289867 0.01113776 0.0052778 ]
 [0.01113776 0.03626638 0.00750979]
 [0.0052778  0.00750979 0.06855407]]


In [75]:
scenarios = 10000

expected_return_simulation = Multivariate_simulation(scenarios, Mean, Cov)
expected_return_simulation

array([[ 0.03795759,  0.24231666,  0.40578675],
       [ 0.12212751,  0.36392429, -0.23549001],
       [ 0.04767978,  0.13255234, -0.71842571],
       ...,
       [-0.11025829, -0.15261385,  0.28909227],
       [-0.09468963, -0.0571538 , -0.02220141],
       [ 0.06221802,  0.26630816,  0.14695437]])

In [76]:
confidence_level = 0.95
require_return = 0.05

# Call the CVaR_optimization_with_Monte_Carlo_approximation function to find the optimal portfolio
__, CVaR_O, portfolio_proportion, Var_O = CVaR_optimization_with_Monte_Carlo_approximation(confidence_level, expected_return_simulation, require_return, short_sell=None)

# Calculate payoffs, VaR, and CVaR using the obtained portfolio proportions
payoffs, VaR, CVaR = payoff(portfolio_proportion, confidence_level, expected_return_simulation)

# Print the results
print('The optimal portfolio proportion is ', dict(zip(Ticker_names,portfolio_proportion)))
print('Mean of return is %f' % np.mean(payoffs))
print('VaR is %f' % -VaR)
print('VaR from optimization is %f' % Var_O)
print('CVaR is %f' % CVaR)
print('CVaR from optimization is %f' % CVaR_O)


The optimal portfolio proportion is  {'AAPL': 0.6700483862232768, 'AMD': 0.25673215876519523, 'BTC-USD': 0.07321945501152997}
Mean of return is 0.050000
VaR is 0.139020
VaR from optimization is 0.139020
CVaR is 0.187170
CVaR from optimization is 0.187170


In [77]:
size = 0.01

hist = go.Figure(data=[go.Histogram(x= payoffs, xbins= dict( # bins used for histogram
                                                start= min(payoffs)//size*size, # type: ignore
                                                end= max(payoffs)//size*size+size, # type: ignore
                                                size= size
                                                            ),)])

hist.update_layout(
    title_text='Portfolio payoffs with mean of return are %f'%np.mean(payoffs), # title of plot
    xaxis_title_text='Return', # xaxis label
    yaxis_title_text='Count', # yaxis label
    # bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)

y_high = [i for i in payoffs if (i>= VaR//size*size and i<= VaR//size*size+size)] # type: ignore
# red line
hist.add_shape(
    dict(
        type="line",
        x0= VaR,
        x1= VaR,
        y0= 0,
        y1= len(y_high),  
        line= dict(
            color= "red",  # Color of the line
            width= 2  # Line width
    )
    )
)
# Add an arrow annotation pointing at the line
arrow_text = 'VaR is %f'%VaR
hist.add_annotation(
    text= arrow_text,
    x= VaR,
    y= len(y_high),  # Adjust the vertical position of the arrow text
    showarrow=True,
    arrowhead=2,  # Arrowhead style (2: arrowhead at the end of the line)
    arrowwidth=2,  # Width of the arrowhead
    arrowcolor="blue",  # Color of the arrowhead
    ax=-30
)

hist.show()

In [78]:
step = 0.001
start = 46
end = 70
CVaR_dict  = CVaR_efficient_forntier(step, start, end, confidence_level, expected_return_simulation, require_return)
CVaR_list = list(CVaR_dict.values())
feasible_return = list(CVaR_dict.keys())

In [79]:
fig = go.Figure(data=go.Scatter(x=CVaR_list, y=feasible_return,name='The efficient frontier'))

fig.update_layout(
    title_text='The efficient frontier', # title of plot
    xaxis_title_text='Risk(CVaR)', # xaxis label
    yaxis_title_text='Return', # yaxis label
    # bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)


<font size="8">CVaR optimization with trading constrain </font>

In [80]:
Ticker_names = ['AAPL', 'AMD', 'TSLA','MSFT','BNB-USD','BTC-USD']
startDate = datetime.datetime(2018, 1, 2)
endDate = datetime.datetime(2021, 12, 31)
interval = '1d'

Close_price_dataframe = pull_data(Ticker_names, startDate, endDate, interval)
Close_price_dataframe

,AAPL,AMD,TSLA,MSFT,BNB-USD,BTC-USD
2018-01-02,40.722870,10.980000,21.368668,80.228989,8.837770,14982.099609
2018-01-03,40.715786,11.550000,21.150000,80.602386,9.535880,15201.000000
2018-01-04,40.904915,12.120000,20.974667,81.311790,9.213990,15599.200195
2018-01-05,41.370621,11.880000,21.105333,82.319916,14.917200,17429.500000
2018-01-08,41.216961,12.280000,22.427334,82.403923,18.260900,15170.099609
...,...,...,...,...,...,...
2021-12-23,174.288605,146.139999,355.666656,328.668762,548.725830,50784.539062
2021-12-27,178.292862,154.360001,364.646667,336.289124,562.641479,50640.417969
2021-12-28,177.264618,153.149994,362.823334,335.110748,534.928040,47588.855469
2021-12-29,177.353592,148.259995,362.063324,335.798187,514.000793,46444.710938


In [81]:
Log_return_datafrme = Log_return(Close_price_dataframe)
Log_return_datafrme

,AAPL,AMD,TSLA,MSFT,BNB-USD,BTC-USD
2018-01-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2018-01-03,-0.000174,0.050610,-0.010286,0.004643,0.076027,0.014505
2018-01-04,0.004634,0.048172,-0.008325,0.008763,-0.034339,0.025858
2018-01-05,0.011321,-0.020001,0.006210,0.012322,0.481792,0.110945
2018-01-08,-0.003721,0.033116,0.060755,0.001020,0.202247,-0.138838
...,...,...,...,...,...,...
2021-12-23,0.003637,0.015585,0.056020,0.004462,0.027868,0.043382
2021-12-27,0.022715,0.054722,0.024935,0.022921,0.025044,-0.002842
2021-12-28,-0.005784,-0.007870,-0.005013,-0.003510,-0.050510,-0.062151
2021-12-29,0.000502,-0.032450,-0.002097,0.002049,-0.039907,-0.024336


In [82]:
shifting_day = 30

Mean, Cov = Mean_Cov(Log_return_datafrme, shifting_day)
print('Mean array is',Mean)
print('Covariance matrix is',Cov)

Mean array is [0.04363721 0.07691238 0.08386873 0.04242019 0.12133154 0.03417301]
Covariance matrix is [[0.01289867 0.01113776 0.01083714 0.00888883 0.00794219 0.0052778 ]
 [0.01113776 0.03626638 0.0153311  0.01056734 0.0102173  0.00750979]
 [0.01083714 0.0153311  0.05010398 0.01024045 0.00997792 0.00733655]
 [0.00888883 0.01056734 0.01024045 0.01061028 0.00781621 0.0052653 ]
 [0.00794219 0.0102173  0.00997792 0.00781621 0.15176906 0.06543545]
 [0.0052778  0.00750979 0.00733655 0.0052653  0.06543545 0.06855407]]


In [83]:
scenarios = 10000

expected_return_simulation = Multivariate_simulation(scenarios, Mean, Cov)
expected_return_simulation

array([[ 0.13336424,  0.01024292,  0.38437485,  0.09688414,  0.55641268,
         0.11413931],
       [ 0.0455559 ,  0.26351138,  0.13678937,  0.02676199, -0.72220126,
        -0.12546518],
       [ 0.0644109 ,  0.40102803, -0.03641968, -0.06030508,  0.3516568 ,
         0.16315341],
       ...,
       [-0.05791511, -0.12248577,  0.02311832,  0.03627661,  0.03904307,
        -0.31227382],
       [ 0.22038109,  0.06493072,  0.08674724,  0.14499168,  0.60745594,
        -0.03165676],
       [ 0.08288426,  0.15027867, -0.10941083,  0.06093189,  0.52367408,
         0.34543355]])

In [84]:
confidence_level = 0.95
require_return = 0.1

# Call the CVaR_optimization_with_Monte_Carlo_approximation function with short selling allowed
__, CVaR_Short, portfolio_proportion_short, Var_short = CVaR_optimization_with_Monte_Carlo_approximation(confidence_level, expected_return_simulation, require_return, short_sell='yes')

# Extract portfolio proportions from the optimization result

# Calculate payoffs, VaR, and CVaR using the obtained portfolio proportions
payoffs, VaR, CVaR = payoff(portfolio_proportion_short, confidence_level, expected_return_simulation)

# Print the results
print('The optimal portfolio proportion is ', dict(zip(Ticker_names,portfolio_proportion_short)))
print('Mean of return is %f' % np.mean(payoffs))
print('VaR is %f' % -VaR)
print('VaR from optimization is %f' % Var_short)
print('CVaR is %f' % CVaR)
print('CVaR from optimization is %f' % CVaR_Short)


The optimal portfolio proportion is  {'AAPL': 0.12775452726828684, 'AMD': 0.42738978656656307, 'TSLA': 0.3344103318654512, 'MSFT': 0.0885984777090493, 'BNB-USD': 0.33219195403370444, 'BTC-USD': -0.3103450774430573}
Mean of return is 0.100000
VaR is 0.189835
VaR from optimization is 0.189835
CVaR is 0.268574
CVaR from optimization is 0.268574


In [85]:
confidence_level = 0.95
require_return = 0.1

# Call the CVaR_optimization_with_Monte_Carlo_approximation function with short selling allowed
__, CVaR_O, portfolio_proportion, Var_O = CVaR_optimization_with_Monte_Carlo_approximation(confidence_level, expected_return_simulation, require_return)

# Extract portfolio proportions from the optimization result

# Calculate payoffs, VaR, and CVaR using the obtained portfolio proportions
payoffs, VaR, CVaR = payoff(portfolio_proportion, confidence_level, expected_return_simulation)

# Print the results
print('The optimal portfolio proportion is ', dict(zip(Ticker_names,portfolio_proportion)))
print('Mean of return is %f' % np.mean(payoffs))
print('VaR is %f' % -VaR)
print('VaR from optimization is %f' % Var_O)
print('CVaR is %f' % CVaR)
print('CVaR from optimization is %f' % CVaR_O)


The optimal portfolio proportion is  {'AAPL': 4.606867533291502e-15, 'AMD': 0.22359576138409262, 'TSLA': 0.32970766027376086, 'MSFT': 4.218333785254949e-15, 'BNB-USD': 0.4466965783421365, 'BTC-USD': 1.065305216013765e-15}
Mean of return is 0.100000
VaR is 0.247520
VaR from optimization is 0.247520
CVaR is 0.341431
CVaR from optimization is 0.341431


In [86]:
hist = go.Figure(go.Bar(x= Ticker_names,
    y=portfolio_proportion, name = 'No-Short'))

hist.add_trace(go.Bar(
    x=Ticker_names,
    y=portfolio_proportion_short,
    name='Short'
    ))

hist.show()


In [87]:
step = 0.001
start = 46
end = 100
CVaR_dict = CVaR_efficient_forntier(step, start, end, confidence_level, expected_return_simulation)
CVaR_dict_S = CVaR_efficient_forntier(step, start, end, confidence_level, expected_return_simulation,short_sell='yes')
CVaR_list, feasible_return = list(CVaR_dict.values()), list(CVaR_dict.keys())
CVaR_list_S, feasible_return_S = list(CVaR_dict_S.values()), list(CVaR_dict_S.keys())

In [88]:
fig = go.Figure(data=go.Scatter(x=CVaR_list, y=feasible_return,name='The portfolio without short-sell'))

fig.update_layout(
    title_text='The efficient frontier', # title of plot
    xaxis_title_text='Risk(CVaR)', # xaxis label
    yaxis_title_text='Return', # yaxis label
    # bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)

fig.add_trace(go.Scatter(x= CVaR_list_S, y= feasible_return_S,name='The portfolio with short-sell'))


fig.show()

<font size="8">CVaR optimization in Option </font>

<font size="4">simulate underlying price</font>

In [89]:
#define variable
Option_dataframe = pd.read_csv('S&P500.csv')
mu=0
theta=0
sigma =0.1206
nu = 0.0031
time_to_maturity = 1/12
spot_price = 295.42
number_of_scenarios = 10000

# Time increment
dt = time_to_maturity

# Shape and scale parameters for gamma distribution
shape = dt / nu
scale = nu

# Generate random samples from the gamma distribution
gamma_rand = np.random.gamma(shape=shape, scale=scale, size=number_of_scenarios)

# Generate random samples from the normal distribution
dW = np.random.normal(size=number_of_scenarios)

# Calculate increments using the Variance Gamma process
ds = sigma * np.sqrt(gamma_rand) * dW

# Calculate log-returns
s = np.log(spot_price) + ds 

# Calculate spot prices at maturity
price_of_underlying_at_maturity = np.exp(s)

<font size="4">Find return for each assets</font>

In [90]:
S_0 = np.array(Option_dataframe['BID'].tolist()+Option_dataframe['ASK'].tolist())

# Get the number of scenarios and assets
number_of_scenarios = len(price_of_underlying_at_maturity)
number_of_asset = len(S_0)

# Calculate the number of options (n)
len_of_vector_z = number_of_asset // 2

# Initialize an empty list to store option payoffs
payoff = []

# Initialize a vector of zeros for calculations
vector_0 = np.array([0 for i in range(number_of_scenarios)])

# Iterate through each option
for i in range(len_of_vector_z):
    
    # Check if the option is a call (ISPUT = 0)
    if Option_dataframe['ISPUT'][i] == 0:
        
        # Calculate call option payoff
        payoff.append(np.maximum(vector_0, price_of_underlying_at_maturity - np.array([Option_dataframe['Strike'][i]] * number_of_scenarios)))
        
    else:
        # Calculate put option payoff
        payoff.append(np.maximum(vector_0, np.array([Option_dataframe['Strike'][i]] * number_of_scenarios) - price_of_underlying_at_maturity))

# Construct the return matrix components 
Ret = [np.array([1] * number_of_scenarios)] + payoff + payoff

# Construct the return matrix DataFrame
Return_dataframe = pd.concat([pd.DataFrame([0 for i in range(number_of_scenarios)]), (pd.DataFrame(payoff + payoff).T - S_0) / S_0], axis=1)

# Set column names for the Return_dataframe
columns = ['Cash'] + ['BID%d' % i for i in Option_dataframe['Strike'].tolist()] + ['ASK%d' % i for i in Option_dataframe['Strike'].tolist()]
Return_dataframe.columns = columns

In [91]:
Return_dataframe

,Cash,BID265,BID270,BID275,BID278,BID279,BID280,BID281,BID282,BID283,...,ASK261,ASK251,ASK252,ASK253,ASK254,ASK256,ASK257,ASK258,ASK259,ASK261
0,0,0.548703,0.615999,0.698820,0.758387,0.774407,0.798254,0.822248,0.847381,0.873770,...,0.480441,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,0,0.016210,-0.002329,-0.033875,-0.062657,-0.077581,-0.090350,-0.105244,-0.122005,-0.140866,...,0.009612,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,0,0.165392,0.170900,0.171395,0.167365,0.161110,0.158600,0.154600,0.149576,0.143392,...,0.141518,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,0,0.324623,0.355798,0.390492,0.412881,0.415879,0.424318,0.431947,0.439450,0.446797,...,0.282310,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,0,-0.113406,-0.152840,-0.212224,-0.262511,-0.284968,-0.306649,-0.331010,-0.357968,-0.387843,...,-0.104995,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,0.084281,0.076715,0.059789,0.042301,0.031333,0.023245,0.013322,0.001916,-0.011160,...,0.069801,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
9996,0,0.926471,1.054662,1.218618,1.340864,1.378836,1.428660,1.480242,1.535096,1.593587,...,0.814464,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
9997,0,-0.572449,-0.685878,-0.843853,-0.970303,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-0.510879,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
9998,0,0.042711,0.028443,0.002589,-0.021796,-0.035180,-0.046126,-0.059086,-0.073762,-0.090370,...,0.033044,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


<font size="4">CVaR optimization </font>

In [92]:
confidence_level = 0.95
beta = confidence_level
q = number_of_scenarios 
number_of_asset = len(S_0) + 1
len_of_vector_z = number_of_asset + 1 + q

# Calculate the mean of mean simulation vector from the Return_dataframe
mean_of_mean_simulation_vector = np.array(Return_dataframe.mean().tolist())

# Set the required return and total wealth
require_return = 4
W = 120000

# Define the decision variable vector z
z = cp.Variable(len_of_vector_z)

# Initialize the objective function coefficients
f = np.zeros(len_of_vector_z)
f[number_of_asset] = 1 
f[number_of_asset+1:] = 1 / (q * (1 - beta))

# Define the objective function for CVaR minimization
objective = cp.Minimize(f.T @ z)

# Create matrices A and Aeq for linear programming
vector_one = np.array([[1 for i in range(q)]])
identity_matrix = np.identity(q)
A = -1 * np.concatenate((Return_dataframe.values, vector_one.T, identity_matrix), axis=1)

# Initialize the right-hand side of the inequalities (b) and equalities (beq)
b = np.zeros(q)
Aeq = np.zeros((2, len_of_vector_z))
Aeq[0, :number_of_asset] = 1
Aeq[1, :number_of_asset] = mean_of_mean_simulation_vector
beq = np.ones(2)
beq[1] = require_return

# Convert matrices A and Aeq to sparse format for efficient storage
A = sparse.csc_matrix(np.asmatrix(A))
Aeq = sparse.csc_matrix(np.asmatrix(Aeq))

bid_constraint = np.array(Option_dataframe['BID_SIZE'].tolist()) * np.array(Option_dataframe['BID'].tolist()) * 100 / W
ask_constraint = np.array(Option_dataframe['ASK_SIZE'].tolist()) * np.array(Option_dataframe['ASK'].tolist()) * 100 / W

Min_bound = np.array([-1*i for i in bid_constraint] + [0 for __ in ask_constraint])
Max_bound = np.array([0 for __ in bid_constraint] + [i for i in ask_constraint])
u_bound = np.zeros(q)
# Define the constraints for CVaR optimization
constraints = [
    A @ z <= b,
    Aeq @ z == beq,
    z[1:number_of_asset] >= Min_bound,
    z[1:number_of_asset] <= Max_bound,
    z[number_of_asset+1:] >= u_bound
]

# Formulate the CVaR optimization problem
problem = cp.Problem(objective, constraints)

# Solve the CVaR optimization problem
problem.solve(solver=cp.SCIPY)

Cvar = problem.value
Var = z[number_of_asset].value
x = z[0:number_of_asset].value

In [93]:
# Calculate the portfolio value based on the optimized portfolio proportion and total wealth
Port = x * W
portfolio_proportion = Port / np.array([1] + list(S_0))

# Compute the payoffs of the portfolio in each scenario
payoffs = sum((portfolio_proportion * np.array(Ret).T).T)

# Calculate Value at Risk (VaR) and Conditional Value at Risk (CVaR)
VaR = np.percentile(payoffs, (1 - confidence_level) * 100)
lower_returns = [i for i in payoffs if i <= VaR]
CVaR = -1 * np.mean(lower_returns)

# Print summary statistics of the portfolio payoffs and risk measures
print('Mean of return is %f' % np.mean(payoffs))
print('VaR is %f' % -VaR)
print('VaR from optimization is %f' % (Var * W - W))
print('CVaR is %f' % CVaR)
print('CVaR from optimization is %f' % (Cvar * W - W))


Mean of return is 600000.000000
VaR is -395303.465095
VaR from optimization is -395303.465095
CVaR is -258889.089610
CVaR from optimization is -258343.432108


In [94]:
size = 10000

hist = go.Figure(data=[go.Histogram(x= payoffs, xbins= dict( # bins used for histogram
                                                start= min(payoffs)//size*size, # type: ignore
                                                end= max(payoffs)//size*size+size, # type: ignore
                                                size= size
                                                            ),)])

hist.update_layout(
    title_text='The histogram of net payoff', # title of plot
    xaxis_title_text='The net payoff(USD)', # xaxis label
    yaxis_title_text='Count', # yaxis label
    # bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)

y_high = [i for i in payoffs if (i>= VaR//size*size and i<= VaR//size*size+size)] # type: ignore
# red line
hist.add_shape(
    dict(
        type="line",
        x0= VaR,
        x1= VaR,
        y0= 0,
        y1= len(y_high),  
        line= dict(
            color= "red",  # Color of the line
            width= 2  # Line width
    )
    )
)
# Add an arrow annotation pointing at the line
arrow_text = 'VaR is %f'%VaR
hist.add_annotation(
    text= arrow_text,
    x= VaR,
    y= len(y_high),  # Adjust the vertical position of the arrow text
    showarrow=True,
    arrowhead=2,  # Arrowhead style (2: arrowhead at the end of the line)
    arrowwidth=2,  # Width of the arrowhead
    arrowcolor="blue",  # Color of the arrowhead
    ax=-30
)

hist.show()

In [95]:
confidence_level = 0.95
beta = confidence_level
q = number_of_scenarios 
number_of_asset = len(S_0) + 1
len_of_vector_z = number_of_asset + 1 + q
CVaR_dict = dict()
Return = [i for i in range(3,13)] 

for i in Return:
    
    # Calculate the mean of mean simulation vector from the Return_dataframe
    mean_of_mean_simulation_vector = np.array(Return_dataframe.mean().tolist())

    # Set the required return and total wealth
    require_return = i
    W = 120000

    # Define the decision variable vector z
    z = cp.Variable(len_of_vector_z)

    # Initialize the objective function coefficients
    f = np.zeros(len_of_vector_z)
    f[number_of_asset] = 1 
    f[number_of_asset+1:] = 1 / (q * (1 - beta))

    # Define the objective function for CVaR minimization
    objective = cp.Minimize(f.T @ z)

    # Create matrices A and Aeq for linear programming
    vector_one = np.array([[1 for i in range(q)]])
    identity_matrix = np.identity(q)
    A = -1 * np.concatenate((Return_dataframe.values, vector_one.T, identity_matrix), axis=1)

    # Initialize the right-hand side of the inequalities (b) and equalities (beq)
    b = np.zeros(q)
    Aeq = np.zeros((2, len_of_vector_z))
    Aeq[0, :number_of_asset] = 1
    Aeq[1, :number_of_asset] = mean_of_mean_simulation_vector
    beq = np.ones(2)
    beq[1] = require_return

    # Convert matrices A and Aeq to sparse format for efficient storage
    A = sparse.csc_matrix(np.asmatrix(A))
    Aeq = sparse.csc_matrix(np.asmatrix(Aeq))

    bid_constraint = np.array(Option_dataframe['BID_SIZE'].tolist()) * np.array(Option_dataframe['BID'].tolist()) * 100 / W
    ask_constraint = np.array(Option_dataframe['ASK_SIZE'].tolist()) * np.array(Option_dataframe['ASK'].tolist()) * 100 / W

    Min_bound = np.array([-1*i for i in bid_constraint] + [0 for __ in ask_constraint])
    Max_bound = np.array([0 for __ in bid_constraint] + [i for i in ask_constraint])
    u_bound = np.zeros(q)
    # Define the constraints for CVaR optimization
    constraints = [
        A @ z <= b,
        Aeq @ z == beq,
        z[1:number_of_asset] >= Min_bound,
        z[1:number_of_asset] <= Max_bound,
        z[number_of_asset+1:] >= u_bound
    ]

    # Formulate the CVaR optimization problem
    problem = cp.Problem(objective, constraints)

    # Solve the CVaR optimization problem
    problem.solve(solver=cp.SCIPY)

    if problem.status == 'optimal':
        
        CVaR_dict[i] = problem.value

In [97]:
CVaR_list = np.array(list(CVaR_dict.values()))*W
feasible_return = np.array(list(CVaR_dict.keys()))*W-W

# Create a scatter plot using Plotly (go.Scatter) for the efficient frontier
fig = go.Figure(data=go.Scatter(x=CVaR_list, y=feasible_return, name='The efficient frontier'))

# Update layout settings for the scatter plot
fig.update_layout(
    title_text='The efficient frontier',  # title of plot
    xaxis_title_text='Risk (CVaR)',  # x-axis label
    yaxis_title_text='Return',  # y-axis label
    bargroupgap=0.1  # gap between bars of the same location coordinates
)


# Show the plot
fig.show()